In [1]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
import nltk
import string
from gensim.models import word2vec
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import torch.nn as nn
import yaml
from tqdm import tqdm
nltk.download('stopwords')
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import re
from torchtext.vocab import GloVe
from torch.nn.utils.rnn import pad_sequence

/home/jazzblazzer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jazzblazzer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset = load_dataset("gigaword")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 3803957
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 189651
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 1951
    })
})


In [3]:
START_TOKEN = "<s>"
END_TOKEN = "</s>"
UNKNOWN_TOKEN = "<unk>"
PAD_TOKEN = "<pad>"

In [4]:
class LSTMDataset:
    def __init__(self, dataset, tokenizer, max_length):
        self.dataset = dataset['document']
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        tokenized_sentence = self.tokenizer(re.sub(r'[^\w\s\']', '',self.dataset[idx].lower()))
        return [START_TOKEN]+tokenized_sentence,tokenized_sentence+[END_TOKEN]

        
    
    
lstm_data = LSTMDataset(dataset['test'],nltk.word_tokenize, 100)
print(lstm_data[0])

(['<s>', 'japan', "'s", 'nec', 'corp', 'and', 'unk', 'computer', 'corp', 'of', 'the', 'united', 'states', 'said', 'wednesday', 'they', 'had', 'agreed', 'to', 'join', 'forces', 'in', 'supercomputer', 'sales'], ['japan', "'s", 'nec', 'corp', 'and', 'unk', 'computer', 'corp', 'of', 'the', 'united', 'states', 'said', 'wednesday', 'they', 'had', 'agreed', 'to', 'join', 'forces', 'in', 'supercomputer', 'sales', '</s>'])


In [5]:
embed_train_data = []
for i in range(len(lstm_data)):
    embed_train_data.append(lstm_data[i][0]+[END_TOKEN])


In [6]:
model = Word2Vec(embed_train_data, vector_size=300, window=5, min_count=3, workers=4)
model.save("word2vec.model")

In [14]:
embedding_model = Word2Vec.load("word2vec.model")
vocab = embedding_model.wv.key_to_index
#change this to random later
vocab[PAD_TOKEN] = len(vocab)-1
vocab[UNKNOWN_TOKEN] = np.random.randint(0,len(vocab))
weights = torch.FloatTensor(embedding_model.wv.vectors)
embeddings = nn.Embedding.from_pretrained(weights)

In [15]:

def collate(batch):
    inputs = []
    targets = []
    for i in range(len(batch)):
        inputs.append(torch.tensor([vocab[word] if word in vocab else vocab[UNKNOWN_TOKEN] for word in batch[i][0]]))
        targets.append(torch.tensor([vocab[word] if word in vocab else vocab[UNKNOWN_TOKEN] for word in batch[i][1]]))
    return pad_sequence(inputs, batch_first=True, padding_value=vocab[PAD_TOKEN]),pad_sequence(targets, batch_first=True, padding_value=vocab[PAD_TOKEN])

In [16]:
lstm_data_loader = DataLoader(lstm_data, batch_size=32, shuffle=True, collate_fn=collate)

In [17]:


if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')



In [18]:
class LM(nn.Module):
    def __init__(self,embedding,batch_size, hidden_size, num_layers,output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size=output_size
        self.batch_size =batch_size
        self.embedding=  embedding
        self.LSTM = nn.LSTM(input_size = embedding.weight.shape[1],hidden_size= hidden_size, num_layers= num_layers,batch_first=True)
        self.output = torch.nn.Linear(hidden_size, output_size)
        
        
    def forward(self, x):
        x= self.embedding(x)
        x, _ = self.LSTM(x)
        x = self.output(x)
        return x

print(embeddings.weight.shape[0])

LanguageModel = LM(embedding=embeddings,
                batch_size=32,
               hidden_size=512,
               num_layers=2,
               output_size=embeddings.weight.shape[0])

2666


In [12]:
# if (config['lm']['train']):
#     LanguageModel = LM(embedding=embedding,
#                         batch_size=config['lm']['batch_size'],
#                         hidden_size=config['lm']['hidden_size'],
#                         num_layers=config['lm']['num_layers'],
#                         output_size=embedding.weight.shape[0])
#     LanguageModel.to(device)
#     criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(LanguageModel.parameters(), lr=config['elmo']['lr'])
#     train_loss=[]
#     test_loss=[]
#     min_val_loss=1e9
#     for e in tqdm(range(config['elmo']['epochs'])):
#         trl,tel=0,0
        
#         c=0
#         for batch in tqdm(TrainDataloader):
#             #N,L
#             x, y= batch
#             x, y = x.to(device), y.to(device)
#             # N, L, C
#             y_pred= LanguageModel(x)
#             y_pred.permute(0,2,1)
#             trloss = criterion(y_pred,y)
#             optimizer.zero_grad()
#             trloss.backward()
#             optimizer.step()
            
#             # tre/=len(sentence)
#             trl+=trloss.item()      
#         trl/=len(TrainDataloader)
#         train_loss.append(trl)
#         c=0
#         for batch in tqdm(ValDataloader):
#             with torch.no_grad():
#                 #N,L
#                 x, y= batch
#                 x, y = x.to(device), y.to(device)
#                 # N, L, C
#                 y_pred= LanguageModel(x)
#                 y_pred.permute(0,2,1)
#                 teloss = criterion(y_pred,y)
#                 tel+=teloss.item()
#         tel/=len(ValDataloader)
#         # tee/=len(val_dataset)
#         test_loss.append(tel)
#         if(tel<min_val_loss):
#             torch.save(LanguageModel,f"../model/elmo_{config['dataset']['name']}.pt")
#     plt.plot(train_loss,label='train')
#     plt.plot(test_loss,label='validation')  
#     plt.legend()
#     plt.savefig(f"LM_{config['lm']['path']}.png")
# else:
#     LanguageModel = torch.load(config['lm']['path']) 


In [30]:
num_epochs = 1

# Lists to store training and validation losses
train_loss = []
val_loss = []
LanguageModel.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(LanguageModel.parameters(), lr=0.9)
c = 1000
for epoch in range(num_epochs):
    LanguageModel.train()
    epoch_train_loss = 0.0
    for batch in lstm_data_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = LanguageModel(inputs)
        outputs = outputs.view(-1, outputs.shape[-1])
        labels = labels.view(-1)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
        print(loss.item())
        c-=1
        if c < 0:
            break
    epoch_train_loss /= len(lstm_data_loader)
    train_loss.append(epoch_train_loss)
    LanguageModel.eval()
    epoch_val_loss = 0.0
    
    # Disable this part of code for now
    # with torch.no_grad():
    #     for batch in val_loader:
    #         inputs, labels = batch
    #         inputs, labels = inputs.to(device), labels.to(device)
    #         outputs = model(inputs)
    #         loss = criterion(outputs, labels)
    #         epoch_val_loss += loss.item()
    # epoch_val_loss /= len(val_loader)
    # val_loss.append(epoch_val_loss)
    
    # if epoch_val_loss < min_val_loss:
    #     torch.save(model.state_dict(), f"model_epoch_{epoch}.pth")

5401.421875
4692.990234375
5370.6826171875
4287.0615234375
4154.22314453125
4113.15966796875
4478.81298828125
5299.26220703125
4309.212890625
4660.97412109375
3949.318359375
4836.50390625
4458.4072265625
3961.369384765625
4316.95458984375
4459.82470703125
4739.6162109375
3695.995849609375
3272.74560546875
4069.689697265625
4324.24609375
4058.249267578125
4226.99658203125
3933.4912109375
3661.6982421875
3441.4990234375
3241.19287109375
3984.089111328125
3611.994873046875
3344.669189453125
4199.4814453125
3783.490234375
3568.78125
3824.4677734375
3109.9716796875
2951.7529296875
3098.111328125
3633.8779296875
3710.006591796875
3544.99755859375
3935.226806640625
3941.65771484375
3216.423828125
3711.536376953125
3791.171875
2800.38427734375
3238.11865234375
3808.598876953125
3554.9814453125
2922.981689453125
3933.182373046875
3088.416015625
2712.734619140625
2987.0380859375
2518.781494140625
3816.980224609375
3768.798095703125
3060.937255859375
3719.266845703125
3351.177734375
3416.41040039

In [31]:
torch.save(LanguageModel, "model.pt")

In [98]:
def SLOR(sentence,embedding_model,vocab,LanguageModel):
    sentence = [START_TOKEN]+nltk.word_tokenize(re.sub(r'[^\w\s\']', '',sentence.lower()))
    labels = sentence[1:]+[END_TOKEN]
    sentence_old = [word if word in vocab else vocab[UNKNOWN_TOKEN] for word in sentence]
    sentence = torch.tensor([vocab[word] if word in vocab else vocab[UNKNOWN_TOKEN] for word in sentence]).squeeze(0)
    labels = torch.tensor([vocab[word] if word in vocab else vocab[UNKNOWN_TOKEN] for word in labels]).squeeze(0)
    LanguageModel.eval()
    with torch.no_grad():
        y_pred= LanguageModel(sentence)
        y_pred = y_pred.squeeze(0)
    y_pred = nn.functional.log_softmax(y_pred,dim=1)
    
    prob_sent = 1
    for i in range(y_pred.shape[0]):
        prob_sent += y_pred[i][labels[i]]
    tot_words = 0
    for keys in vocab.keys():
        tot_words+=embedding_model.wv.get_vecattr(keys,'count')
    unigram_prob = 1
    for i in range(len(sentence)):
        unigram_prob *= embedding_model.wv.get_vecattr(sentence_old[i],'count')/tot_words
    return (prob_sent)/(len(sentence)+1) - np.log(unigram_prob)
    

In [99]:
SLOR(dataset['test']['document'][0],embedding_model,vocab,LanguageModel)

tensor(-3150.1748)